In [21]:

from __future__ import print_function
import numpy as np
from keras.layers import Input, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, LSTM, Reshape, Lambda
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model, Sequential
from keras import backend as K
from keras.layers  import TimeDistributed, TimeDistributedDense
import os, sys, re
from nltk import tokenize
import nltk
nltk.download('punkt')
import string 
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shivamverma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import sys
print(sys.version)

2.7.9 (v2.7.9:648dcafa7e5f, Dec 10 2014, 10:10:46) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
TEXT_DATA_DIR = './speech_data/data/'

In [5]:
# Load the labels 
df = pd.read_csv('./speech_data/presidents_meta.csv')
label_dict = dict(zip(list(df.foldername), list(df.label)))

In [6]:
texts = []  # list of text samples
# labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids


for name in sorted(os.listdir(TEXT_DATA_DIR)):

    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        for fname in sorted(os.listdir(path)):
            if 'combined' not in fname and 'combines' not in fname:
                president_name = path.split('/')[-1]
                fpath = os.path.join(path, fname)
                f = open(fpath)
                if (president_name in label_dict):
                    labels.append(label_dict[president_name])
                    line = f.read()
                    line = re.sub(r'[^\x00-\x7F]+',' ', line).lower()
                    texts.append(line)
                f.close()
                

print('Found %s texts.' % len(texts))
print('Found %s texts.' % len(labels))

Found 862 texts.
Found 862 texts.


In [7]:
texts[500][:1000]

'the president. good afternoon, ladies and gentlemen.i have been asked to give a statement about the consular convention that is pending before the united states senate.i should like to say very briefly that i hope the senate will give its advice and consent to the proposed convention with the u.s.s.r. i feel very strongly that the ratification of this treaty is very much in our national interest. i feel this way for two principal reasons:first, we need this treaty to protect 18,000 american citizens who each year travel from this country to the soviet the convention requires immediate notification to us whenever an american is arrested in the soviet union. it insures our right to visit that citizen within 4 and as often thereafter as is desirable.we think that we need these rights help to protect american citizens. these are rights which the soviet citizens already have who travel in this country, because guaranteed by our constitution.second, the convention does not require the openi

In [9]:
MAX_NB_WORDS = 100
print(len(texts))
mini_test = texts[:10]
print(list(map(len, mini_test)))
MAX_SEQUENCE_LENGTH = 100
MAX_SENTENCES = 100

862
[2665, 11080, 10013, 1927, 81929, 62451, 51770, 54751, 5014, 6845]


In [11]:
def pad_dense_list(speech, maxsentence):
    
    each_sub_len = len(speech[0])
    Z = np.zeros((maxsentence, each_sub_len))
    #print("Shape of Z{}, list {}", Z.shape, len(speech))
    for index, row in enumerate(speech):
        if (index >= maxsentence):
            break
        
        Z[index] = row
    #print(Z) 
    return Z

In [12]:
def preprocess_text(mini_test):
    #mini_test = "mini test. hello world."
    tokenizer = Tokenizer(nb_words=5)
    tokenizer.fit_on_texts(mini_test)
    text_encoded = np.zeros((len(texts),MAX_SENTENCES,MAX_SEQUENCE_LENGTH))
    #print(text_encoded.shape)
    word_index = tokenizer.word_index
    for enu, speech in enumerate(mini_test):
        sents = tokenize.sent_tokenize(speech)
        if(len(sents) == 0):
            continue
        sequences = tokenizer.texts_to_sequences(sents)
        
        #print(len(sequences))
        #print('Found %s unique tokens.' % len(word_index))
        data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
        text_encoded[enu] = (pad_dense_list(data, 100))
    return word_index, text_encoded
word_index, text_encoded = preprocess_text(texts)

In [13]:
# First Speech encoding
data = text_encoded
labels = np.asarray(labels)
print(data.shape)
print(labels.shape)

(862, 100, 100)
(862,)


In [14]:
new_data = data
data = data.reshape(data.shape[0], data.shape[1]*data.shape[1])

In [15]:
data.shape

(862, 10000)

In [80]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
# labels = labels[indices]
VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of data tensor: (862, 10000)
Shape of label tensor: (862,)


In [17]:
embeddings_index = {}
GLOVE_DIR = '../../glove.6B/'
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [66]:
EMBEDDING_DIM = 100

In [67]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [68]:
data.shape

(862, 10000)

In [35]:
# y_train = y_train.reshape((-1, 1))

In [77]:
# NUM_SAMPLES = 32
# VOCAB_SIZE = 1000
# EMBEDDING_DIM = 64
# MAX_SEQUENCE_LENGTH = 10
# MAX_SENTENCES = 20
# EMBEDDING_INPUT_LENGTH = MAX_SEQUENCE_LENGTH*MAX_SENTENCES
# WORD_LSTM_DIM = 100
# SENT_LSTM_DIM = 100
# NUM_CLASSES = 7

# input_array = np.random.randint(1000, size=(32, 20*10)) #batch = 32, sent = 20, words = 10
# x_train = input_array[:25]
# x_val = input_array[25:]
# y_train = np.random.randint(1,7,25)
# y_val = np.random.randint(1,7,7)
# print(x_train.shape)
# print(y_train.shape)
# print(x_val.shape)
# print(y_val.shape)
# embedding_matrix = np.random.randint(0,999,(1000,64))

In [70]:
VOCAB_SIZE = len(word_index) + 1
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 100
MAX_SENTENCES = 100
EMBEDDING_INPUT_LENGTH = MAX_SEQUENCE_LENGTH*MAX_SENTENCES
WORD_LSTM_DIM = 100
SENT_LSTM_DIM = 100
NUM_CLASSES = 7

In [89]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 
                            EMBEDDING_DIM, 
                            weights = [embedding_matrix],
                            input_length=EMBEDDING_INPUT_LENGTH,
                            trainable=False)
model.add(embedding_layer)
model.add(Reshape((MAX_SENTENCES, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM,), input_shape=(EMBEDDING_INPUT_LENGTH,EMBEDDING_DIM,)))
# model.add(TimeDistributed(LSTM(WORD_LSTM_DIM, input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM), return_sequences=True)))
model.add(Lambda(lambda x: K.mean(x, axis=2, keepdims=True), output_shape=lambda s: (s[0], 1, s[1],s[3])))
model.add(TimeDistributed(LSTM(SENT_LSTM_DIM, input_shape=(MAX_SENTENCES, EMBEDDING_DIM), return_sequences=True)))
model.add(Reshape((MAX_SENTENCES,SENT_LSTM_DIM), input_shape=(1,MAX_SENTENCES,SENT_LSTM_DIM)))
model.add(Lambda(lambda x: K.mean(x, axis=1, keepdims=True), output_shape=lambda s: (s[0], s[2])))
model.add(Reshape((SENT_LSTM_DIM,)))
model.add(Dense(NUM_CLASSES, input_dim=SENT_LSTM_DIM, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_28 (Embedding)         (None, 10000, 100)    0           embedding_input_26[0][0]         
____________________________________________________________________________________________________
reshape_46 (Reshape)             (None, 100, 100, 100) 0           embedding_28[0][0]               
____________________________________________________________________________________________________
lambda_28 (Lambda)               (None, 1, 100, 100)   0           reshape_46[0][0]                 
____________________________________________________________________________________________________
timedistributed_22 (TimeDistribu (None, 1, 100, 100)   80400       lambda_28[0][0]                  
___________________________________________________________________________________________

In [83]:
x_train.shape

(690, 10000)

In [88]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=5, batch_size=10)

Exception: Error when checking model target: expected dense_13 to have shape (None, 7) but got array with shape (690, 1)